In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab 

from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

#read csv into dataframe
GOPSentiDf = pd.read_csv('2016GOPPresDebSenti.csv')
AirlineSentiDf = pd.read_csv('TweetsUSAirlineSenti.csv')

#Begin feature removal
GOPSentiDf.count()

id                           13871
candidate                    13775
candidate_confidence         13871
relevant_yn                  13871
relevant_yn_confidence       13871
sentiment                    13871
sentiment_confidence         13871
subject_matter               13545
subject_matter_confidence    13871
candidate_gold                  28
name                         13871
relevant_yn_gold                32
retweet_count                13871
sentiment_gold                  15
subject_matter_gold             18
text                         13871
tweet_coord                     21
tweet_created                13871
tweet_id                     13871
tweet_location                9959
user_timezone                 9468
dtype: int64

In [2]:
AirlineSentiDf.count()

tweet_id                        14640
airline_sentiment               14640
airline_sentiment_confidence    14640
negativereason                   9178
negativereason_confidence       10522
airline                         14640
airline_sentiment_gold             40
name                            14640
negativereason_gold                32
retweet_count                   14640
text                            14640
tweet_coord                      1019
tweet_created                   14640
tweet_location                   9907
user_timezone                    9820
dtype: int64

In [3]:
#we want the sentiment analysis to be as general as possible, independent of user location, when they tweeted, 
#the tweet's subject matter, how often their tweets get retweeted, who is the user, 

GOPSentiDf.drop('user_timezone', 1, inplace = True)
GOPSentiDf.drop('tweet_location', 1, inplace = True)
GOPSentiDf.drop('tweet_id', 1, inplace = True)
GOPSentiDf.drop('tweet_created', 1, inplace = True)
GOPSentiDf.drop('tweet_coord', 1, inplace = True)
#subject matter gold is the specific topic of tweeted text, such as Religion, Abortion, Immigration, FOX news, etc.
GOPSentiDf.drop('subject_matter_gold', 1, inplace = True)
#sentiment gold is repetitive of the sentiment column
GOPSentiDf.drop('sentiment_gold', 1, inplace = True)
GOPSentiDf.drop('retweet_count', 1, inplace = True)
#relevant_yn_gold is repetitive of relevant
GOPSentiDf.drop('relevant_yn_gold', 1, inplace = True)
GOPSentiDf.drop('name', 1, inplace = True)
#candidate_gold is repetitive of candidate
GOPSentiDf.drop('candidate_gold', 1, inplace = True)
GOPSentiDf.drop('subject_matter_confidence', 1, inplace = True)
GOPSentiDf.drop('subject_matter', 1, inplace = True)
GOPSentiDf.drop('relevant_yn_confidence', 1, inplace = True)
GOPSentiDf.drop('relevant_yn', 1, inplace = True)
GOPSentiDf.drop('candidate_confidence', 1, inplace = True)
GOPSentiDf.drop('candidate', 1, inplace = True)

#we want to do the same for the tweets for the US Airlines
AirlineSentiDf.drop('user_timezone', 1, inplace = True)
AirlineSentiDf.drop('tweet_location', 1, inplace = True)
AirlineSentiDf.drop('tweet_created', 1, inplace = True)
AirlineSentiDf.drop('tweet_coord', 1, inplace = True)
AirlineSentiDf.drop('retweet_count', 1, inplace = True)
AirlineSentiDf.drop('negativereason_gold', 1, inplace = True)
AirlineSentiDf.drop('name', 1, inplace = True)
AirlineSentiDf.drop('airline_sentiment_gold', 1, inplace = True)
AirlineSentiDf.drop('airline', 1, inplace = True)
AirlineSentiDf.drop('negativereason_confidence', 1, inplace = True)
AirlineSentiDf.drop('negativereason', 1, inplace = True)

In [4]:
GOPSentiDf.count()

id                      13871
sentiment               13871
sentiment_confidence    13871
text                    13871
dtype: int64

In [11]:
AirlineSentiDf.count()

tweet_id                        14640
airline_sentiment               14640
airline_sentiment_confidence    14640
text                            14640
dtype: int64

In [13]:
#create features: number of negative and positive words and emoticons, number of emoticons
GOPTxtAr = GOPSentiDf.values[0:,][:,3]
GOPNumPosAr = []
GOPNumNegAr = []
GOPNumEmoAr = []
for i in range(len(GOPTxtAr)):
    GOPNumPosAr.append(0)
    GOPNumNegAr.append(0)
    GOPNumEmoAr.append(0)
    
AirTxtAr = AirlineSentiDf.values[0:,][:,3]
AirNumPosAr = []
AirNumNegAr = []
AirNumEmoAr = []

for in in range(len(AirTxtAr)):
    AirNumPosAr.append(0)
    AirNumNegAr.append(0)
    AirNumEmoAr.append(0)
    
index = 0
for text in GOPTxtAr:
    #remove indefinite and definite articles
    GOPTxtAr[index] = str(GOPTxtAr[index]).replace('The ', '')
    GOPTxtAr[index] = str(GOPTxtAr[index]).replace('the ', '')
    GOPTxtAr[index] = str(GOPTxtAr[index]).replace('An ', '')
    GOPTxtAr[index] = str(GOPTxtAr[index]).replace('an ', '')
    GOPTxtAr[index] = str(GOPTxtAr[index]).replace('A ', '')
    GOPTxtAr[index] = str(GOPTxtAr[index]).replace('a ', '')
    index += 1

index = 0
for text in AirTxtAr:
    AirTxtAr[index] = str(AirTxtAr[index]).replace('The ', '')
    AirTxtAr[index] = str(AirTxtAr[index]).replace('the ', '')
    AirTxtAr[index] = str(AirTxtAr[index]).replace('An ', '')
    AirTxtAr[index] = str(AirTxtAr[index]).replace('an ', '')
    AirTxtAr[index] = str(AirTxtAr[index]).replace('A ', '')
    AirTxtAr[index] = str(AirTxtAr[index]).replace('a ', '')
    index += 1

RT @NancyLeeGrahn: How did everyone feel about the Climate Change question last night? Exactly. #GOPDebate
RT @NancyLeeGrahn: How did everyone feel about Climate Change question last night? Exactly. #GOPDebate
RT @ScottWalker: Didn't catch the full #GOPdebate last night. Here are some of Scott's best lines in 90 seconds. #Walker16 http://t.co/ZSfF…
RT @ScottWalker: Didn't catch full #GOPdebate last night. Here are some of Scott's best lines in 90 seconds. #Walker16 http://t.co/ZSfF…
RT @TJMShow: No mention of Tamir Rice and the #GOPDebate was held in Cleveland? Wow.
RT @TJMShow: No mention of Tamir Rice and #GOPDebate was held in Cleveland? Wow.
RT @RobGeorge: That Carly Fiorina is trending -- hours after HER debate -- above any of the men in just-completed #GOPdebate says she's on …
RT @RobGeorge: That Carly Fiorinis trending -- hours after HER debate -- above any of men in just-completed #GOPdebate says she's on …
RT @DanScavino: #GOPDebate w/ @realDonaldTrump delivered the highest r

Although the occasional dick, @realDonaldTrump sure knows how to keep these political debates alive and exciting. #GOPDebate
Although occasional dick, @realDonaldTrump sure knows how to keep these political debates alive and exciting. #GOPDebate
RT @LaurenGreenberg: So... it's okay to kill people if you're in the military, but it's not okay to get an abortion if you're a rape victim…
RT @LaurenGreenberg: So... it's okay to kill people if you're in military, but it's not okay to get abortion if you're rape victim…
RT @NYMag: Hillary Clinton spent #GOPDebate night with the Kardashians: http://t.co/gkLcCJoBi5 http://t.co/76XRSJ0rNQ
RT @NYMag: Hillary Clinton spent #GOPDebate night with Kardashians: http://t.co/gkLcCJoBi5 http://t.co/76XRSJ0rNQ
So, it's official; the GOP 2016 Platform is less abortion, more war. That's it. That's what they are after. #GOPDebate #Priorities
So, it's official; GOP 2016 Platform is less abortion, more war. That's it. That's what they are after. #GOPDebate #Pr

&gt;@realDonaldTrump has now spent more time attacking @megynkelly on Twitter thhe spent on any #GOPdebate subject. http://t.co/L9WWPiEHdI
RT @fuzzytypewriter: THE CHOICE IS CLEAR #GOPDebate #FiskOwlsley2016 http://t.co/UcGHGu2IPu
RT @fuzzytypewriter: THE CHOICE IS CLEAR #GOPDebate #FiskOwlsley2016 http://t.co/UcGHGu2IPu
Rubio Budget Raises MiddleClass Taxes, Adds $4.5T Deficit, Cuts InvestmentTax To $0 http://t.co/9S9NuVSAf7 #GOPDebate http://t.co/z9v38Vhi03
Rubio Budget Raises MiddleClass Taxes, Adds $4.5T Deficit, Cuts InvestmentTax To $0 http://t.co/9S9NuVSAf7 #GOPDebate http://t.co/z9v38Vhi03
RT @bourgeoisalien: Can't wait until the end of the debate when they wrestle an angry, meth addicted bear to the death to see who the real …
RT @bourgeoisalien: Can't wait until end of debate when they wrestle angry, meth addicted bear to death to see who real …
RT @BRios82: Yes, @JebBush Obama's dividing the country by creating jobs, ending wars, achieving diplomacy w/ Iran &amp; giving peop

RT @rickbot5000: Our news media is broken #GOPDebate
RT @rickbot5000: Our news mediis broken #GOPDebate
#MikeHuckabee should drop a mixtape called "Kill People; Break Things" and feature @WakaFlocka on the title track #GOPDebate
#MikeHuckabee should drop mixtape called "Kill People; Break Things" and feature @WakaFlockon title track #GOPDebate
RT @HelenGym2015: Love Twitter bringing humor to #GOPdebate but end of day 2015 GOP is still the anti-woman, anti-immigrant, pro-rich, anti…
RT @HelenGym2015: Love Twitter bringing humor to #GOPdebate but end of day 2015 GOP is still anti-woman, anti-immigrant, pro-rich, anti…
RT @billmaher: Good for you, Megyn Kelly, calling Ben Carson an idiot right off the bat. #GOPDebate
RT @billmaher: Good for you, Megyn Kelly, calling Ben Carson idiot right off bat. #GOPDebate
RT @benwikler: So, was @SenSchumer throwing his hat in the #GOPDebate ring by coming out on the GOP side of the #IranDeal debate tonight?
RT @benwikler: So, was @SenSchumer throwing h

@SpeakerBoehner curious You say @POTUS used executive decisions against the law, but on #GOPDebate every one said they'd repeal? Isn't same?
@SpeakerBoehner curious You say @POTUS used executive decisions against law, but on #GOPDebate every one said they'd repeal? Isn't same?
RT @GayPatriot: Hey @JebBush - American citizenship is NOT a universal human right. 

#NoMoreBushes #GOPDebate
RT @GayPatriot: Hey @JebBush - Americcitizenship is NOT universal humright. 

#NoMoreBushes #GOPDebate
A debate with no losers: @GOP should be proud | http://t.co/koWXyZUI4z  #GOPDebate #KidsTable #GOP2016 #TCOT #WakeUpAmerica
debate with no losers: @GOP should be proud | http://t.co/koWXyZUI4z  #GOPDebate #KidsTable #GOP2016 #TCOT #WakeUpAmerica
@rushlimbaugh yep @megynkelly sucked at #GOPDebate  @realDonaldTrump  country is in a mess &amp; debate questions Terrible
@rushlimbaugh yep @megynkelly sucked at #GOPDebate  @realDonaldTrump  country is in mess &amp; debate questions Terrible
#GOPDebate winners

...

RT @jamiaw: "The purpose of the military is to kill people and break things"-Mike Huckabee #GOPDebates #KKKorGOP
RT @jamiaw: "purpose of military is to kill people and break things"-Mike Huckabee #GOPDebates #KKKorGOP
RT @LindaSuhler: Jeb Stands By "Act of Love"; Comments and Earned Legal Status [BOO]
#NoAmnesty
http://t.co/hImScrVikr
#GOPDebates
RT @LindaSuhler: Jeb Stands By "Act of Love"; Comments and Earned Legal Status [BOO]
#NoAmnesty
http://t.co/hImScrVikr
#GOPDebates
RT @Apathycase: So, Trump buys politicians, used bankruptcy to his advantage, skirted laws for money, and admitted to wanting single payer …
RT @Apathycase: So, Trump buys politicians, used bankruptcy to his advantage, skirted laws for money, and admitted to wanting single payer …
RT @RWSurferGirl: I'm really really really pissed off at FOX News for what they did to @realDonaldTrump #GOPDebate  #GOPDebates 🇺🇸
RT @RWSurferGirl: I'm really really really pissed off at FOX News for what they did to @realDonaldTrump #GO

Did you notice @ScottWalker nodding in agreement when @BenCarson talked about operating on brain? #GOPdebates #LZDebates
RT @RWSurferGirl: It is very disappointing that Fox News is not conducting a "Fair &amp; Balanced" Debate. 🇺🇸 #GOPDebate  #GOPDebates
RT @RWSurferGirl: It is very disappointing that Fox News is not conducting "Fair &amp; Balanced" Debate. 🇺🇸 #GOPDebate  #GOPDebates
RT @RWSurferGirl: Why should @realDonaldTrump pledge support to the GOP when the establishment sold out to Obama? #GOPDebate  #GOPDebates 🇺🇸
RT @RWSurferGirl: Why should @realDonaldTrump pledge support to GOP when establishment sold out to Obama? #GOPDebate  #GOPDebates 🇺🇸
RT @NateMJensen: Walker: God is cutting the UW budget. Talk to him. #GOPdebates
RT @NateMJensen: Walker: God is cutting UW budget. Talk to him. #GOPdebates
RT @SupermanHotMale: Dr. Carson talking about race is like trump talking about money. #Wrong #GopDebates
RT @SupermanHotMale: Dr. Carson talking about race is like trump talking about

Wish they would ask more questions of @SenTedCruz #GOPdebates
RT @RWSurferGirl: Ask Trump a legitimate question. Look at Wallace's face when Trump nails it. 🇺🇸 #GOPDebate  #GOPDebates
RT @RWSurferGirl: Ask Trump legitimate question. Look at Wallace's face when Trump nails it. 🇺🇸 #GOPDebate  #GOPDebates
Why is everybody orange? #GOPDebates #GOPDebate
Why is everybody orange? #GOPDebates #GOPDebate
RT @RWSurferGirl: Why doesn't Chris Wallace ask the other politicans about their finances and where their money comes from? 🇺🇸 #GOPDebate  …
RT @RWSurferGirl: Why doesn't Chris Wallace ask other politicans about their finances and where their money comes from? 🇺🇸 #GOPDebate  …
I wonder if any of these candidates actually have an idea other than "Obama is an idiot" #GOPDebates
I wonder if any of these candidates actually have ideother th"Obamis idiot" #GOPDebates
RT @RWSurferGirl: You would never know @realDonaldTrump  is the frontrunner from watching this debate. 🇺🇸 #GOPDebate  #GOPDebates
RT 

Okay GOP candidates if anyone if you have uterus you cspeak on abortions, if not STFU! #GOPDebates #StandwithPP
RT @DWSTwit: Jeb Bush "deciding" what women should do with their bodies! #GOPdebates
RT @DWSTwit: Jeb Bush "deciding" what women should do with their bodies! #GOPdebates
RT @RWSurferGirl: I think Cruz and Trump need to band together and expose this set up job, and get rid of Bush and Rubio, 🇺🇸 #GOPDebate  #G…
RT @RWSurferGirl: I think Cruz and Trump need to band together and expose this set up job, and get rid of Bush and Rubio, 🇺🇸 #GOPDebate  #G…
Planned parenthood coming up... I apologize in advance for any super drunk twitter status updates...  #GOPDebates #FOXNEWSDEBATE
Planned parenthood coming up... I apologize in advance for any super drunk twitter status updates...  #GOPDebates #FOXNEWSDEBATE
RT @monaeltahawy: Oh boy: "social issues" = GOP code for who can control women's bodies &amp; sexuality the most. Hold on tight, TwitterLand  #…
RT @monaeltahawy: Oh boy: "social

RT @AnnemarieWeers: @PoetinPoeville Did you know there are aprx 700 laws governing women's bodies and not one governing men's? It's true.
#…
RT @AnnemarieWeers: @PoetinPoeville Did you know there are aprx 700 laws governing women's bodies and not one governing men's? It's true.
#…
Trump - "I had the good sense to take MY money and run. Screw y'all" #GOPDebates
Trump - "I had good sense to take MY money and run. Screw y'all" #GOPDebates
RT @marymauldin: Dig into everyone's financials if you are doing that to #Trump ! 
#GOPDebates
RT @marymauldin: Dig into everyone's financials if you are doing that to #Trump ! 
#GOPDebates
RT @goldietaylor: Huckabee. #GOPDebates http://t.co/Zy12bdUkx9
RT @goldietaylor: Huckabee. #GOPDebates http://t.co/Zy12bdUkx9
RT @ncsr18: Muppet show. #GOPdebates
RT @ncsr18: Muppet show. #GOPdebates
FYI...I'm watching the debates on DVR.   Here comes my posts and tweets.  #GOPDebates
FYI...I'm watching debates on DVR.   Here comes my posts and tweets.  #GOPDebates
"T

In [ ]:
#Begin feature creation
#attribute for number of positive words

#attribute for number of negative words

#attribute for number of positive emoticons

#attribute for number of negative emoticons

